# trial code for graph comm

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from scipy.sparse import csgraph
# Load the TNBC dataset
adata = sc.read_h5ad("C:/Users/zzrtroy/OneDrive/umich/2025 Fall/BIOINF593/final_project/data/bc_dataset.h5ad")
# Check the structure of the AnnData object
print(adata)

AnnData object with n_obs × n_vars = 42512 × 28200
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mito', 'subtype', 'celltype_subset', 'celltype_minor', 'celltype_major', 'ident', 'label', 'n_counts', 'cell_type', 'size_factors'
    var: 'n_cells', 'feature_name', 'hvg', 'hvg_score'
    uns: '_from_cache', 'ccc_target', 'data_reference', 'data_url', 'dataset_description', 'dataset_id', 'dataset_name', 'dataset_organism', 'dataset_reference', 'dataset_summary', 'dataset_url', 'knn', 'normalization_id', 'pca_variance', 'target_organism', 'var_names_all'
    obsm: 'X_pca'
    varm: 'pca_loadings'
    layers: 'counts', 'normalized'
    obsp: 'knn_connectivities', 'knn_distances'


In [4]:
# Ensure we are working with the main expression matrix
# Normalization and log transformation steps
if 'normalized' in adata.layers:
    # If a normalized layer exists, use it
    adata.X = adata.layers['normalized']
else:
    # Normalize total counts
    sc.pp.normalize_total(adata, target_sum=1e4, inplace=True)  # Normalize the main data matrix
    sc.pp.log1p(adata)  # Log transformation

# Extract the expression matrix for further analysis
X = adata.X

# Calculate the pairwise distances
distance_matrix = pairwise_distances(X)

# Construct the graph using a distance threshold for connectivity
threshold = 0.5  # Set this threshold according to your analysis needs
adjacency_matrix = (distance_matrix < threshold).astype(int)

# Create the graph from the adjacency matrix
G = nx.from_numpy_array(adjacency_matrix)

# Optionally, compute graph metrics (e.g., number of edges, nodes)
num_edges = G.number_of_edges()
num_nodes = G.number_of_nodes()
print(f'Number of nodes: {num_nodes}, Number of edges: {num_edges}')

# Perform community detection using the Girvan-Newman algorithm
def girvan_newman(G):
    while G.number_of_edges() > 0:
        # Find the edge with the highest betweenness centrality
        edge_betweenness = nx.edge_betweenness_centrality(G)
        edge_to_remove = max(edge_betweenness, key=edge_betweenness.get)
        G.remove_edge(*edge_to_remove)

        # Check connected components to find communities
        components = list(nx.connected_components(G))
        if len(components) > 1:
            return components

# Apply the Girvan-Newman algorithm
communities = girvan_newman(G)

# Map nodes to communities
community_map = {node: idx for idx, community in enumerate(communities) for node in community}

# Assign community information back to the AnnData object
adata.obs['community'] = [community_map[i] for i in range(num_nodes)]

# Visualization of the communities
plt.figure(figsize=(10, 8))
nx.draw(G, node_color=list(community_map.values()), with_labels=False, node_size=50)
plt.title('Cell Communication Network')
plt.show()

# Further analysis: Average expression per community
average_expression = {}
for community_id in set(community_map.values()):
    member_indices = [i for i, cid in community_map.items() if cid == community_id]
    average_expression[community_id] = np.mean(X[member_indices], axis=0)

# Prepare a DataFrame of average expression
average_expression_df = pd.DataFrame(average_expression)

# Plot average expression
average_expression_df.plot(kind='bar', figsize=(12, 6))
plt.title('Average Expression per Community')
plt.xlabel('Genes/Cytokines')
plt.ylabel('Average Expression')
plt.show()

KeyboardInterrupt: 

In [ ]:
nx.from_numpy_array